In [1]:
# Copyright 2017 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from os import path
import csv
import numpy as np
import tensorflow as tf

from tensorflow.contrib.timeseries.python.timeseries import estimators as ts_estimators
from tensorflow.contrib.timeseries.python.timeseries import model as ts_model
from tensorflow.contrib.timeseries.python.timeseries import NumpyReader

import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt


class _LSTMModel(ts_model.SequentialTimeSeriesModel):
    def __init__(self,num_units, num_features, dtype=tf.float32):
        super(_LSTMModel, self).__init__(
        # Pre-register the metrics we'll be outputting (just a mean here).
        train_output_names=["mean"],
        predict_output_names=["mean"],
        num_features=num_features,
        dtype=dtype)
        self._num_units = num_units
        self._lstm_cell = None
        self._lstm_cell_run = None
        self._predict_from_lstm_output = None

    def initialize_graph(self, input_statistics):
        super(_LSTMModel, self).initialize_graph(input_statistics=input_statistics)
        self._lstm_cell = tf.nn.rnn_cell.LSTMCell(num_units=self._num_units)
    # Create templates so we don't have to worry about variable reuse.
        self._lstm_cell_run = tf.make_template(
        name_="lstm_cell",
        func_=self._lstm_cell,
        create_scope_now_=True)
    # Transforms LSTM output into mean predictions.
        self._predict_from_lstm_output = tf.make_template(
        name_="predict_from_lstm_output",
        func_=lambda inputs: tf.layers.dense(inputs=inputs, units=self.num_features),
        create_scope_now_=True)

    def get_start_state(self):
        return (tf.zeros([], dtype=tf.int64),tf.zeros([self.num_features], dtype=self.dtype),[tf.squeeze(state_element, axis=0) for state_element in self._lstm_cell.zero_state(batch_size=1, dtype=self.dtype)])

    def _transform(self, data):
        mean, variance = self._input_statistics.overall_feature_moments
        return (data - mean) / variance

    def _de_transform(self, data):
        mean, variance = self._input_statistics.overall_feature_moments
        return data * variance + mean

    def _filtering_step(self, current_times, current_values, state, predictions):
        
        state_from_time, prediction, lstm_state = state
        with tf.control_dependencies(
            [tf.assert_equal(current_times, state_from_time)]):
            transformed_values = self._transform(current_values)
      # Use mean squared error across features for the loss.
        predictions["loss"] = tf.reduce_mean(
          (prediction - transformed_values ) ** 2, axis=-1)
      # Keep track of the new observation in model state. It won't be run
      # through the LSTM until the next _imputation_step.
        new_state_tuple = (current_times,transformed_values , lstm_state)
        return (new_state_tuple, predictions)

    def _prediction_step(self, current_times, state):
        _, previous_observation_or_prediction, lstm_state = state
        lstm_output, new_lstm_state = self._lstm_cell_run(
        inputs=previous_observation_or_prediction, state=lstm_state)
        next_prediction = self._predict_from_lstm_output(lstm_output)
        new_state_tuple = (current_times, next_prediction, new_lstm_state)
        return new_state_tuple, {"mean": self._de_transform(next_prediction)}
    def _imputation_step(self, current_times, state):
    # Does not do anything special if we're jumping across a gap. More advanced
    # models, especially probabilistic ones, would want a special case that
    # depends on the gap size.
        return state

    def _exogenous_input_step(self, current_times, current_exogenous_regressors, state):
        raise NotImplementedError("Exogenous inputs are not implemented for this example.")


if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    with open('temperature.csv','r',encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        temperature = [float(row['ano']) for row in reader]
    y = np.array(temperature)
    x = np.arange(1,137,1)


    data = {
      tf.contrib.timeseries.TrainEvalFeatures.TIMES: x,
      tf.contrib.timeseries.TrainEvalFeatures.VALUES: y,
    }

    reader = NumpyReader(data)

    train_input_fn = tf.contrib.timeseries.RandomWindowInputFn(
      reader, batch_size=6, window_size=25)

    estimator = ts_estimators.TimeSeriesRegressor(
      model=_LSTMModel(num_features=1, num_units=128),
      optimizer=tf.train.AdamOptimizer(0.001))

    estimator.train(input_fn=train_input_fn, steps=20000)
    evaluation_input_fn = tf.contrib.timeseries.WholeDatasetInputFn(reader)
    evaluation = estimator.evaluate(input_fn=evaluation_input_fn, steps=1)

    (predictions,) = tuple(estimator.predict(
      input_fn=tf.contrib.timeseries.predict_continuation_input_fn(
          evaluation, steps=10)))

    observed_times = evaluation["times"][0]
    observed = evaluation["observed"][0, :, :]
    evaluated_times = evaluation["times"][0]
    evaluated = evaluation["mean"][0]
    predicted_times = predictions['times']
    predicted = predictions["mean"]
    plt.figure(figsize=(15, 5))
    observed_lines = plt.plot(observed_times, observed, label="observation", color="k")
    evaluated_lines = plt.plot(evaluated_times, evaluated, label="evaluation", color="g")
    predicted_lines = plt.plot(predicted_times, predicted, label="prediction", color="r")
    plt.legend(handles=[observed_lines[0], evaluated_lines[0], predicted_lines[0]],
             loc="upper left")
    plt.savefig('predict_result.jpg')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmp67i8sh49', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002A0B82D7588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMINI

INFO:tensorflow:global_step/sec: 50.483
INFO:tensorflow:loss = 0.380823, step = 7301 (1.981 sec)
INFO:tensorflow:global_step/sec: 74.1714
INFO:tensorflow:loss = 0.590706, step = 7401 (1.349 sec)
INFO:tensorflow:global_step/sec: 69.1959
INFO:tensorflow:loss = 0.350489, step = 7501 (1.445 sec)
INFO:tensorflow:global_step/sec: 71.5205
INFO:tensorflow:loss = 0.656985, step = 7601 (1.462 sec)
INFO:tensorflow:global_step/sec: 49.1194
INFO:tensorflow:loss = 0.484845, step = 7701 (1.972 sec)
INFO:tensorflow:global_step/sec: 74.3367
INFO:tensorflow:loss = 0.384931, step = 7801 (1.345 sec)
INFO:tensorflow:global_step/sec: 74.1713
INFO:tensorflow:loss = 0.518833, step = 7901 (1.348 sec)
INFO:tensorflow:global_step/sec: 73.7886
INFO:tensorflow:loss = 0.943117, step = 8001 (1.355 sec)
INFO:tensorflow:global_step/sec: 53.2502
INFO:tensorflow:loss = 0.37352, step = 8101 (1.878 sec)
INFO:tensorflow:global_step/sec: 72.1914
INFO:tensorflow:loss = 0.369431, step = 8201 (1.384 sec)
INFO:tensorflow:global

INFO:tensorflow:loss = 0.393579, step = 15601 (1.946 sec)
INFO:tensorflow:global_step/sec: 71.0129
INFO:tensorflow:loss = 0.313121, step = 15701 (1.408 sec)
INFO:tensorflow:global_step/sec: 73.1947
INFO:tensorflow:loss = 0.837035, step = 15801 (1.365 sec)
INFO:tensorflow:global_step/sec: 72.6632
INFO:tensorflow:loss = 0.393351, step = 15901 (1.376 sec)
INFO:tensorflow:global_step/sec: 52.4683
INFO:tensorflow:loss = 0.633679, step = 16001 (1.906 sec)
INFO:tensorflow:global_step/sec: 70.5623
INFO:tensorflow:loss = 0.530023, step = 16101 (1.418 sec)
INFO:tensorflow:global_step/sec: 73.2483
INFO:tensorflow:loss = 0.321843, step = 16201 (1.365 sec)
INFO:tensorflow:global_step/sec: 71.9319
INFO:tensorflow:loss = 0.304549, step = 16301 (1.390 sec)
INFO:tensorflow:global_step/sec: 52.8564
INFO:tensorflow:loss = 0.547171, step = 16401 (1.893 sec)
INFO:tensorflow:global_step/sec: 59.2756
INFO:tensorflow:loss = 0.331356, step = 16501 (1.685 sec)
INFO:tensorflow:global_step/sec: 64.0571
INFO:tenso

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADMINI~1\AppData\Local\Temp\tmp67i8sh49\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
